# Adversarial Autoencoder with K-means on CIFAR-10

In [ ]:
import torch
import numpy as np
from torchvision import datasets
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.autograd import Variable


# to import CIFAR-10 as torch tensor
transform = transforms.ToTensor()

# load the training and test datasets
train_data = datasets.CIFAR10(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.CIFAR10(root='data', train=False,
                                  download=True, transform=transform)

## training and testing dataloaders

In [ ]:
# Create training and test dataloaders

num_workers = 0
# how many samples per batch to load
batch_size = 100
# how many epochs for training
num_epochs = 25
# latent space size
z_dim = 50

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

## model design

In [ ]:
# create an encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encode = nn.Sequential(
            nn.Linear(32*32*3, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(True),
            nn.Linear(1000, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(True),
            nn.Linear(500, z_dim)
        )

    def forward(self, x):
        x = x.view(-1, 32*32*3)
        out = self.encode(x)
        return out

In [ ]:
# create a decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decode = nn.Sequential(
            nn.Linear(z_dim, 500),
            nn.ReLU(True),
            nn.Linear(500, 1000),
            nn.ReLU(True),
            nn.Linear(1000, 32*32*3),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.decode(input)
        return output.view(-1, 3, 32, 32)

In [ ]:
# create a discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(z_dim, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(True),
            nn.Linear(1000, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(True),
            nn.Linear(500, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.dis(x)
        return x

## CAE model instantiation

In [ ]:
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()


optim_dc = torch.optim.Adam(discriminator.parameters(), lr=1e-3)
optim_de = torch.optim.Adam(decoder.parameters(), lr=1e-3)
optim_en = torch.optim.Adam(encoder.parameters(), lr=1e-3)

# set loss function
criterion = nn.MSELoss()
dc_criterion = nn.BCEWithLogitsLoss()
gn_criterion = nn.BCEWithLogitsLoss()

## load wieghts

## training

In [ ]:
## training
for x, y in iter(train_loader):
    images = x
grid = torchvision.utils.make_grid(images[-64:])
tb = SummaryWriter()
tb.add_image('images', grid)

x = 0

for epoch in range(num_epochs):
    start_time = time.time()
    total_loss = 0
    dc_total_loss = 0
    gn_total_loss = 0

    for data in (train_loader):
        x += 1
        img, _ = data
        # Autoencoder training
        embed = encoder(img)
        output = decoder(embed)
        loss = criterion(output, img)
        # back propagation
        optim_en.zero_grad()
        optim_de.zero_grad()
        loss.backward()
        optim_en.step()
        optim_de.step()
        total_loss += loss.item()
        tb.add_scalar('Loss ae/step', loss.item(), x)
        
        
        # discriminator training
        for i in range(4):
            # Generating data
            real_distribution = torch.randn(batch_size, z_dim)
            encoder_output = encoder(img).detach()
            # GAN training
            d_real = discriminator(real_distribution)
            d_fake = discriminator(encoder_output)
            optim_dc.zero_grad()
            dc_loss_real = dc_criterion(d_real, Variable(torch.ones(batch_size, 1)))
            dc_loss_real.backward()
            dc_loss_fake = dc_criterion(d_fake, Variable(torch.zeros(batch_size, 1)))
            dc_loss_fake.backward()
            
            optim_dc.step()
        dc_total_loss += 0.5 * (dc_loss_real.item() + dc_loss_fake.item())
        tb.add_scalar('Loss dc/step', 0.5*(dc_loss_real.item() + dc_loss_fake.item()), x)
        # generator training
        optim_en.zero_grad()
        encoder_output = encoder(img)
        d_fake = discriminator(encoder_output)
        gn_loss = gn_criterion(d_fake, Variable(torch.ones(batch_size, 1)))
        
        gn_loss.backward()
        optim_en.step()
        gn_total_loss += gn_loss.item()
        tb.add_scalar('Loss gn/step', gn_loss.item(), x)
    
    # tensorboard stuff
    grid = torchvision.utils.make_grid(output[-64:])
    tb.add_image('images', grid)
    tb.add_histogram('test histo', embed, epoch)    
    tb.add_scalar('Loss ae', total_loss/(50000/batch_size), epoch)
    tb.add_scalar('Loss dc', dc_total_loss/(50000/batch_size), epoch)
    tb.add_scalar('Loss gn', gn_total_loss/(50000/batch_size), epoch)
    
    end_time = time.time()
    # log
    print('epoch [{}/{}], loss dc:{:.4f}, loss gn:{:.4f}, loss ae:{:.4f}, expected runtime:{:.4f} min'
          .format(epoch+1, num_epochs, 
                  dc_total_loss/(50000/batch_size), gn_total_loss/(50000/batch_size), 
                  total_loss/(50000/batch_size), (end_time - start_time)*num_epochs/60))

## visualizing reconstructed results

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
encoder.eval()
decoder.eval()

# get sample outputs
with torch.no_grad():
    output = decoder(encoder(images))
    
# print images and output in tensorborad
grid = torchvision.utils.make_grid(images[:64])
tb.add_image('real', grid)
grid = torchvision.utils.make_grid(output[:64])
tb.add_image('reconstructed', grid)

## reconstruct the train dataset

In [ ]:
# run the dataset through the trained model
encoder.eval()
decoder.eval()

for image_index, data in enumerate(train_loader):
    images, itr_labels = data
    # forward pass: compute embedded outputs by passing inputs to the model
    encoded = encoder(images)
    decoded = decoder(encoded)
    if not image_index:
        #First group encoded in new array
        embedded = encoded.detach().numpy()
        #first group decoded
        disembedded = decoded.detach().numpy()
        #labels
        labels = itr_labels.detach().numpy()
        continue
    #stacking the remaining data
    embedded = np.vstack((embedded, encoded.detach().numpy()))
    disembedded = np.vstack((disembedded, decoded.detach().numpy()))
    labels = np.hstack((labels, itr_labels.detach().numpy()))

## clustering model instantiation and training

In [ ]:
from sklearn.cluster import KMeans

# create a k-means model to cluster the embedded features
clustering_model = KMeans(n_clusters=10, tol = 1e-4, max_iter = 400).fit(embedded)

## visualizing results and performance evaluation

### testing autoencoder

In [ ]:
# run the dataset through the trained model
encoder.eval()
decoder.eval()

for image_index, data in enumerate(test_loader):
    images, itr_labels = data
    # forward pass: compute embedded outputs by passing inputs to the model
    encoded = encoder(images)
    decoded = decoder(encoded)    
    if not image_index:
        #First group encoded in new array
        embedded_test = encoded.detach().numpy()
        #first group decoded
        disembedded_test = decoded.detach().numpy()
        #labels
        labels_test = itr_labels.detach().numpy()
        continue
    #stacking the remaining data
    embedded_test = np.vstack((embedded_test, encoded.detach().numpy()))
    disembedded_test = np.vstack((disembedded_test, decoded.detach().numpy()))
    labels_test = np.hstack((labels_test, itr_labels.detach().numpy()))

In [ ]:
labels_pred = clustering_model.predict(embedded_test)

In [ ]:
import import_ipynb
import DataVisuals as dv


view_results = dv.DataVisuals(disembedded.reshape(-1, 3, 32,32), labels, clustering_model.labels_)
view_results_test = dv.DataVisuals(disembedded_test.reshape(-1, 3, 32, 32), labels_test, labels_pred)

In [ ]:
# confusion matrix of training set
view_results.cm()

In [ ]:
# confusion matrix of testing set
view_results_test.cm()

In [ ]:
# scatter plot of subset of training set
view_results.scat(embedded)

In [ ]:
# scatter plot of subset of testing set
view_results_test.scat(embedded_test)

In [ ]:
# train metrics
metric = dv.Metrics(labels, clustering_model.labels_)
nmi = metric.nmi()
ari = metric.ari()
acc = metric.acc()
print('NMI = {:.4f} \nARI = {:.4f} \nACC = {:.4f}'.format(nmi, ari, acc))

In [ ]:
# test metrics
metric = dv.Metrics(labels_test, labels_pred)
nmi = metric.nmi()
ari = metric.ari()
acc = metric.acc()
print('NMI = {:.4f} \nARI = {:.4f} \nACC = {:.4f}'.format(nmi, ari, acc))

## store wieghts